<span style='font-size:x-large'>Enunciado.</span>

1. Diseñar un algoritmo que permita calcular el MCD para polinomios multivariados con dos variables.



In [1]:
Para hallar el MCD de polinomios con dos variables se sigue el **Algoritmo Euclídeo extendido**:

1. Elegir una variable principaL.
2. Descomponer cada polinomio como polinomio en x con coeficientes en y.
3. Calcular el contenido (máximo común divisor de coeficientes).
4. Dividir por el contenido → parte primitiva.
5. Aplicar pseudo-división entre las partes primitivas.
6. Recalcular el contenido del resto y dividir.
7. Repetir hasta que el resto sea cero.
8. El MCD final es: `gcd(contents) * gcd(primitive parts)`.

<span style='font-size:x-large'>Desarrollo en Python a detalle </span>


In [1]:
from collections import defaultdict

def clean_poly(p):
    return {k: v for k, v in p.items() if v != 0}

def leading_term(poly):
    return max(poly.keys(), key=lambda exp: (exp[0], exp[1]))

def poly_add(p1, p2):
    result = defaultdict(int)
    for exp, coeff in p1.items():
        result[exp] += coeff
    for exp, coeff in p2.items():
        result[exp] += coeff
    return clean_poly(result)

def poly_sub(p1, p2):
    result = defaultdict(int)
    for exp, coeff in p1.items():
        result[exp] += coeff
    for exp, coeff in p2.items():
        result[exp] -= coeff
    return clean_poly(result)

def poly_mul(p1, p2):
    result = defaultdict(int)
    for (e1x, e1y), c1 in p1.items():
        for (e2x, e2y), c2 in p2.items():
            result[(e1x+e2x, e1y+e2y)] += c1 * c2
    return clean_poly(result)

def poly_divmod_x(f, g):
    """Divide f entre g respecto a x (como univariado en x)"""
    q = defaultdict(int)
    r = f.copy()
    while r and max(exp[0] for exp in r) >= max(exp[0] for exp in g):
        lx_r = leading_term(r)
        lx_g = leading_term(g)
        dx = lx_r[0] - lx_g[0]
        dy = lx_r[1] - lx_g[1]
        if dy != 0 or dx < 0:
            break
        coeff = r[lx_r] / g[lx_g]
        term = {(dx, 0): coeff}
        q = poly_add(q, term)
        to_subtract = poly_mul(term, g)
        r = poly_sub(r, to_subtract)
    return clean_poly(q), clean_poly(r)

def poly_gcd(f, g):
    """Calcula el MCD de dos polinomios multivariados usando Euclides"""
    while g:
        _, r = poly_divmod_x(f, g)
        f, g = g, r
    # Normalizar: dividir por coeficiente líder
    lt = leading_term(f)
    coeff = f[lt]
    return {exp: c / coeff for exp, c in f.items()}

SyntaxError: invalid non-printable character U+00A0 (607009015.py, line 58)

<span style='font-size:x-large'>SageMath “con funciones predefinidas”</span>


In [3]:
Ry.<y> = PolynomialRing(QQ)     # Anillo QQ[y]
S.<x>  = PolynomialRing(Ry)     # Anillo (QQ[y])[x]


In [17]:
def gcd_multivariado(u, v, var, verbose=False):
    """
    MCD de u, v en QQ[y][x], tomando 'var' como variable principal.
    """
    if u.is_zero(): 
        return v
    if v.is_zero(): 
        return u

    cont_u = gcd(u.coefficients())
    cont_v = gcd(v.coefficients())
    g_cont = gcd(cont_u, cont_v)

    pp_u = u // cont_u
    pp_v = v // cont_v

    A, B = pp_u, pp_v

    while not B.is_zero():
        quo, rem = A.pseudo_quo_rem(B)
        if rem.is_zero():
            A = B
            break
        cont_r = gcd(rem.coefficients())
        pp_r   = rem // cont_r
        A, B   = B, pp_r

    return (g_cont * A).factor()


In [16]:
# Ejemplo A: gcd((1/2)x^2*y + (3/4)x*y^2, (1/4)x*y) = 1/4*x*y
uA = S((1/2)*x^2*y + (3/4)*x*y^2)
vA = S((1/4)*x*y)
print("Ejemplo 1 :  gcd((1/2)x^2*y + (3/4)x*y^2, (1/4)x*y) = ", gcd_multivariado(uA, vA, x))

# Ejemplo B: gcd((2/3)x^2 + (4/9)x*y, (8/15)x^2 + (16/45)x*y) = 2/9*x
uB = S((2/3)*x^2 + (4/9)*x*y)
vB = S((8/15)*x^2 + (16/45)*x*y)
print("Ejemplo 2 : gcd((2/3)x^2 + (4/9)x*y, (8/15)x^2 + (16/45)x*y) = ", gcd_multivariado(uB, vB, x))

# Ejemplo C: gcd(x/2 + y/3, x/4 + y/6) = 1/12*(3*x + 2*y)
uC = S(x/2 + y/3)
vC = S(x/4 + y/6)
print("Ejemplo 3 : gcd(x/2 + y/3, x/4 + y/6) = ", gcd_multivariado(uC, vC, x))


Ejemplo 1 :  gcd((1/2)x^2*y + (3/4)x*y^2, (1/4)x*y) =  (1/4) * y * x
Ejemplo 2 : gcd((2/3)x^2 + (4/9)x*y, (8/15)x^2 + (16/45)x*y) =  (8/45) * x * (3*x + 2*y)
Ejemplo 3 : gcd(x/2 + y/3, x/4 + y/6) =  (1/12) * (3*x + 2*y)


In [15]:
# Ejemplo 1: u = x^2*y + x*y^2, v = x*y  
u1 = S(x^2*y + x*y^2)
v1 = S(x*y)
g1 = gcd_multivariado(u1, v1, x)
show(f"Ejemplo 4 : gcd(x^2*y + x*y^2, x*y) = {g1}")   

# Ejemplo 2: u = x^2 + 2*x*y + y^2, v = x^2 - y^2  
u2 = S(x^2 + 2*x*y + y^2)
v2 = S(x^2 - y^2)
g2 = gcd_multivariado(u2, v2, x)
show(f"Ejemplo 5 : gcd(x^2 + 2xy + y^2, x^2 - y^2) = {g2}")   

# Ejemplo 3: u = x^3*y^2 + x^2*y^3, v = x^2*y^2  
u3 = S(x^3*y^2 + x^2*y^3)
v3 = S(x^2*y^2)
g3 = gcd_multivariado(u3, v3, x)
show(f"Ejemplo 6 : gcd(x^3*y^2 + x^2*y^3, x^2*y^2) = {g3}")   

'Ejemplo 4 : gcd(x^2*y + x*y^2, x*y) = y * x'

'Ejemplo 5 : gcd(x^2 + 2xy + y^2, x^2 - y^2) = (2) * (x + y)'

'Ejemplo 6 : gcd(x^3*y^2 + x^2*y^3, x^2*y^2) = y^2 * x^2'

In [14]:
# Ejemplo 4: 
# u = (x + y + 1)*(x^2 - y),   v = (x + y + 1)*(x - y^2)

u4 = S((x + y + 1)*(x^2 - y))
v4 = S((x + y + 1)*(x - y^2))
g4 = gcd_multivariado(u4, v4, x)
show(f"Ejemplo 7 : gcd = {g4}")   # esperado: x + y + 1

# Ejemplo 5: 
# u = x^3*y^3 + 2*x*y^3 + y^3,   v = x^2*y^3 - y^3

u5 = S(x^3*y^3 + 2*x*y^3 + y^3)
v5 = S(x^2*y^3 - y^3)
g5 = gcd_multivariado(u5, v5, x)
show(f"Ejemplo 8 : gcd = {g5}")   

# Ejemplo 6: 
# u = (x^2 + 3*x*y + 2*y^2)*(x - 2*y),  
# v = (x^2 + 3*x*y + 2*y^2)*(x + y)

base = x^2 + 3*x*y + 2*y^2
u6   = S(base * (x - 2*y))
v6   = S(base * (x + y))
g6   = gcd_multivariado(u6, v6, x)
show(f"Ejemplo 9 : gcd = {g6}")  


'Ejemplo 7 : gcd = x + y + 1'

'Ejemplo 8 : gcd = -8*y^3'

'Ejemplo 9 : gcd = (-3) * (x + y) * (x + 2*y)'